# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
display(data)
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [3]:
len(data[data['days_employed'].isna()])

2174

In [4]:
len(data[data['total_income'].isna()])

2174

# **Вывод**

Из 12 столбцов датафрейма два содержат пропуски: общий трудовой стаж в днях (2174) и ежемесячный доход (2174). Эти же столбцы имеют вещественный тип данных. Некоторые их них являются отрицательными числами, что свидетельствует о техногенной природе ошибки. 2174 строки - существенный объем данных. Чтобы не исказить результат, их можно заменить медианными значениями для каждого столбца.

## Шаг 2. Предобработка данных

### Обработка пропусков

Переведем из датафрейма все отрицательные числа в натуральные, чтобы они не повлияли на медиану, а затем заменим на соответствующее медианное значение все пропуски в столбцах. 

In [5]:
data['days_employed'] = abs(data['days_employed'])
data['total_income'] = abs(data['total_income'])

In [6]:
data['days_employed'] = data['days_employed'].fillna(data.groupby('dob_years')['days_employed'].transform('median'))
len(data[data['days_employed'].isna()])

0

In [7]:
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))
len(data[data['total_income'].isna()])

0

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В датафрейме не осталось пропущенных значений.

### Замена типа данных

In [9]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

В Датафрейме теперь все числовые значения имеют вид целочисленных.

### Обработка дубликатов

In [10]:
data.duplicated().sum()

54

In [11]:
duplicated_data = data[data.duplicated()]
display(duplicated_data.head(54))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1864,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
4182,1,1615,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,349414,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,331245,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
7808,0,333953,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,331245,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
9238,2,1615,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
9528,0,363827,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9627,0,6850,56,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции со своей недвижимостью
10462,0,354981,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,покупка коммерческой недвижимости


В столбце education_id встречаются одинаковые по смыслу значения, но записанные в разном регистре. Приведем их к нижнему регистру. Так как в датафреме не все значения строчные, то сделаем это циклом, который избежит ошибки атрибута.

In [12]:
for col in data.columns:
    try:
        data[col] = data[col].str.lower()
    except AttributeError:
        pass

In [13]:
data.duplicated().sum()

71

Удяляем явные дубликаты, так как 71 - число, не повлияющее на результаты исследования.

In [14]:
data = data.drop_duplicates().reset_index()
print('Количество  дубликатов:', data.duplicated().sum())

Количество  дубликатов: 0


**Вывод**

Обнаруженное количество дубликатов удалено методом сброса старых индексов. 

### Лемматизация

Сначала выведем все уникальные значения столбца с целями кредита.

In [15]:
purposes = data['purpose'].unique()
print(purposes)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Цели отчетливо делятся на 4 типа: операции с недвижимостью, приобретение автомобиля, получение образования и организация свадьбы. Создадим функцию, которая поделит все заявки на данные 4 типа, основываясь на лемме в значении столбца с целью получения кредита. Полученные категории сохраним в качестве нового столбца справа, который формируется методом apply() и применяет функцию к значениям столбца 'purpose'.

In [16]:
from pymystem3 import Mystem
m = Mystem()
def create_category_purpose(row):
    lem_purpose = m.lemmatize(row['purpose'])
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose ):
        return 'недвижимость'
    if 'образование' in lem_purpose:
        return 'образование'
    if 'свадьба' in lem_purpose:
        return 'свадьба'

data['purpose_category'] = data.apply(create_category_purpose, axis=1)

In [17]:
print(data.head(15))

    index  children  days_employed  dob_years            education  \
0       0         1           8437         42               высшее   
1       1         1           4024         36              среднее   
2       2         0           5623         33              среднее   
3       3         3           4124         32              среднее   
4       4         0         340266         53              среднее   
5       5         0            926         27               высшее   
6       6         0           2879         43               высшее   
7       7         0            152         50              среднее   
8       8         2           6929         35               высшее   
9       9         0           2188         41              среднее   
10     10         2           4171         36               высшее   
11     11         0            792         40              среднее   
12     12         0         357103         65              среднее   
13     13         0 

**Вывод**

Лемматизация помогла категоризовать получателей кредита по целям, значения которых выражены в строчном формате.

### Категоризация данных

Разобьем данные на несколько категорий в зависимости от целей исследования:
 - по количеству детей (бездетные, 1 ребенок, больше 1 ребенка)

In [18]:
def create_child_category(row):
    if row == 0:
        return 'бездетные'
    if row == 1:
        return '1 ребенок'
    if row > 1:
        return 'больше 1 ребенка'
data['child_category'] = data['children'].apply(create_child_category)
print(data.head(15))

    index  children  days_employed  dob_years            education  \
0       0         1           8437         42               высшее   
1       1         1           4024         36              среднее   
2       2         0           5623         33              среднее   
3       3         3           4124         32              среднее   
4       4         0         340266         53              среднее   
5       5         0            926         27               высшее   
6       6         0           2879         43               высшее   
7       7         0            152         50              среднее   
8       8         2           6929         35               высшее   
9       9         0           2188         41              среднее   
10     10         2           4171         36               высшее   
11     11         0            792         40              среднее   
12     12         0         357103         65              среднее   
13     13         0 

**Вывод**

Чтобы выявлять закономерности, необходимо классифицировать заемщиков по разным типам в зависимости от целей исследования: по критерию наличия детей, по семейному статусу и уровню дохода.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок

Разделив заявителей на 3 категории по атрибуту children, сгруппируем отдельную таблицу, а затем построим сводную таблицу, отображающую количество выплаченных вовремя и просроченных кредитов для каждой из категорий:

In [19]:
children_debt = data.loc[:, ['children', 'debt','child_category']]
display(children_debt.head(15))

,children,debt,child_category
0,1,0,1 ребенок
1,1,0,1 ребенок
2,0,0,бездетные
3,3,0,больше 1 ребенка
4,0,0,бездетные
5,0,0,бездетные
6,0,0,бездетные
7,0,0,бездетные
8,2,0,больше 1 ребенка
9,0,0,бездетные


In [20]:
pivot_children_debt = children_debt.pivot_table(index='child_category', columns='debt', values='children', aggfunc='count')
display(pivot_children_debt.head(15))

debt,0,1
child_category,,
1 ребенок,4364,444
бездетные,13028,1063
больше 1 ребенка,2275,233


для наглядности необходимо добавить к данным столбец с процентным соотношением количества просрочек к общему количеству заемов:

In [21]:
pivot_children_debt['percent'] = (pivot_children_debt[1]/(pivot_children_debt[1] + pivot_children_debt[0])) * 100
display(pivot_children_debt)


debt,0,1,percent
child_category,,,
1 ребенок,4364,444,9.234609
бездетные,13028,1063,7.543822
больше 1 ребенка,2275,233,9.290271


**Вывод**

Зависимость между наличием детей и выплатой кредита в срок присутствует.

Создадим сводную таблицу, демонстрирующую количество выплаченных в срок кредитов и просроченных по атрибуту семейного положения:

In [22]:
pivot_relationship_debt = data.pivot_table(index='family_status', columns='debt', values='gender', aggfunc='count')
display(pivot_relationship_debt)

debt,0,1
family_status,,
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931
не женат / не замужем,2536,274


In [23]:
relationship_debt = data.loc[:, ['family_status', 'debt','relationship_stutus']]
display(relationship_debt.head(15))

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,family_status,debt,relationship_stutus
0,женат / замужем,0,NaN
1,женат / замужем,0,NaN
2,женат / замужем,0,NaN
3,женат / замужем,0,NaN
4,гражданский брак,0,NaN
5,гражданский брак,0,NaN
6,женат / замужем,0,NaN
7,женат / замужем,0,NaN
8,гражданский брак,0,NaN
9,женат / замужем,0,NaN


In [24]:
pivot_relationship_debt['percent'] = (pivot_relationship_debt[1]/(pivot_relationship_debt[1] + pivot_relationship_debt[0])) * 100
display(pivot_relationship_debt)

debt,0,1,percent
family_status,,,
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182
не женат / не замужем,2536,274,9.750890


**Вывод**

Заемщики, не состоящие в узаконенных отношениях, имеют самый высокий процент просрочек по кредиту - свыше 9 %. Самыми ответственными являются овдовевшие граждане, среди которых "штрафников" на целый процент меньше, чем среди женатых!

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

По доходу заемщики будут делиться на 3 категории относительно общей медианы столбца: 
- низкий (ниже медианного диапазона);
- средний (внутри медианного диапазона и выше медианного диапазона, то есть в промежутке  от 100000 до 180000);
- высокий (выше медианного диапазона)
Для начала определим медиану столбца доходов:

In [25]:
data['total_income'].median()

142594.0

In [26]:
def create_income_status_category(row):
    if row < 100000:
        return 'низкий'
    if row > 180000:
        return 'высокий'
    else:
        return 'средний'
data['income_status'] = data['total_income'].apply(create_income_status_category)
print(data.head(50))

    index  children  days_employed  dob_years            education  \
0       0         1           8437         42               высшее   
1       1         1           4024         36              среднее   
2       2         0           5623         33              среднее   
3       3         3           4124         32              среднее   
4       4         0         340266         53              среднее   
5       5         0            926         27               высшее   
6       6         0           2879         43               высшее   
7       7         0            152         50              среднее   
8       8         2           6929         35               высшее   
9       9         0           2188         41              среднее   
10     10         2           4171         36               высшее   
11     11         0            792         40              среднее   
12     12         0         357103         65              среднее   
13     13         0 

In [27]:
income_debt = data.loc[:, ['income_status', 'debt','total_income']]
display(income_debt.head(15))

,income_status,debt,total_income
0,высокий,0,253875
1,средний,0,112080
2,средний,0,145885
3,высокий,0,267628
4,средний,0,158616
5,высокий,0,255763
6,высокий,0,240525
7,средний,0,135823
8,низкий,0,95856
9,средний,0,144425


In [28]:
pivot_income_debt = income_debt.pivot_table(index='income_status', columns='debt', values='total_income', aggfunc='count')
display(pivot_income_debt)

debt,0,1
income_status,,
высокий,5945,478
низкий,4109,354
средний,9659,909


In [29]:
pivot_income_debt['percent'] = (pivot_income_debt[1]/(pivot_income_debt[1] + pivot_income_debt[0])) * 100
display(pivot_income_debt)

debt,0,1,percent
income_status,,,
высокий,5945,478,7.442005
низкий,4109,354,7.931884
средний,9659,909,8.601438


**Вывод**

Самая высокий процент просрочек среди тех, чей ежемесячный доход определен как средний. Процент просрочек среди тех, чей доход выше или ниже приблизительно равен. Вывод: уровень дохода не сильно влияет на возможность невыплаты кредита в срок. Это зависит от других факторов (личной ответственности или способности идти на риск: меньше рискуют те, кто стеснен в средствах или, наоборот, те, кому есть что терять).

- Как разные цели кредита влияют на его возврат в срок?

По целям заемщики уже были категоризированы в п.2.4

In [30]:
pivot_purpose = data.pivot_table(index=['purpose_category'], columns = 'debt', values = 'gender', aggfunc= 'count')
display(pivot_purpose)

debt,0,1
purpose_category,,
автомобиль,3903,403
недвижимость,10029,782
образование,3643,370
свадьба,2138,186


In [31]:
pivot_purpose['percent'] = (pivot_purpose[1]/(pivot_purpose[1] + pivot_purpose[0])) * 100
display(pivot_purpose)

debt,0,1,percent
purpose_category,,,
автомобиль,3903,403,9.359034
недвижимость,10029,782,7.233373
образование,3643,370,9.220035
свадьба,2138,186,8.003442


**Вывод**

Самый большой процент просрочек среди тех, кто берет кредит на автомобиль и образование. Самыми надёжными заемщиками оказываются те, кто за счет кредитных средств совершает операции с недвижимостью.

## Шаг 4. Общий вывод

Исходя из проведенного анализа, можно составить портрет предполагаемого "надёжного" заемщика:
-без детей;
-вдовец/вдова;
-целью кредита является приобретение недвижимости

Уровень дохода неординарно влияет на вероятность невыплаты в срок. Как выяснилось, люди с доходом выше среднего и ниже среднего равным образом могут просрочить платежи, тогда как у заемщикоd с медианным уровнем дохода эта вероятность выше.